# 4 OAD updates

In this project,
we seek to optimize an aircraft of approximately 150 passengers that flies at Mach 0.78,
using different engine and fuel technologies.

## The OAD problem

The OAD problem is to find the values of the design parameters 
that minimize the maximum take-off mass (MTOM, `mtom` in the code) under some operation constraints.
Note that the MTOM of an airplane is a good criterion to optimize the design.
Alternative objectives are the cash operating cost (COC, `coc` in the code) or the direct operating cost (DOC, `doc` in the code).

### The design parameters

The **design parameters** are :

- the maximum sea level static thrust  (100 kN ≤ slst ≤ 200 kN, default: 150 kN),
- the number of passengers  (120 ≤ n_pax ≤ 180, default: 150),
- the wing area (100 m² ≤ area ≤ 200 m², default: 180 m²),
- the wing aspect ratio  (5 ≤ ar ≤ 20, default: 9.).

### The operational constraints

The **operational constraints** are :

- the take-off field length (tofl ≤ 1900 m),
- the approach speed (vapp ≤ 135 kt),
- the vertical speed (300 ft/min ≤ vz),
- the wing span (span ≤ 40 m),
- the wing length (length ≤ 45 m),
- the fuel margin (0% ≤ fm).

## The uncertain parameters

In Problems 2 and 3,
some parameters are considered as uncertain.

These uncertain parameters are modelled as random variables defined by probability distributions:

| Variable | Distribution          | Fuel type | Engine type |
|----------|-----------------------|-----------|-------------|
| `gi`     | T(0.35, 0.4, 0.405)   | liquid_h2 | All         |
| `vi`     | T(0.755, 0.800, 0805) | liquid_h2 | All         |
| `aef`    | T(0.99, 1., 1.03)     | All       | All         |
| `cef`    | T(0.99, 1., 1.03)     | All       | All         |
| `aef`    | T(0.99, 1., 1.03)     | All       | All         |
| `fc_pwd` | T(0.8, 1, 1.02)       | liquid_h2 | electrofan  |
| `bed`    | U(400,700)            | battery   | All         |

where `T(minimum, mode, maximum)` and `U(minimum,maximum)`
represent the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution)
and the [uniform distribution](https://en.wikipedia.org/wiki/Continuous_uniform_distribution) respectively.
the last two columns indicate the type of fuel and the type of engine
for which these uncertain parameters are relevant.

The parameters `aef`, `cef` and `aef` are related
to the three main technical areas involved in aircraft design,
namely aerodynamics, propulsion and structure.
The lower, the better.
These factors are representing the unknown
included in any creative activity.
Their probability distributions are not symmetrical
as it is always easier to make something less efficient than expected...

## The 12 models

The 12 models are defined in `gemseo-oad` as Python functions:



In [1]:
from gemseo_oad_training.models import aerodynamic
from gemseo_oad_training.models import approach
from gemseo_oad_training.models import battery
from gemseo_oad_training.models import climb
from gemseo_oad_training.models import engine
from gemseo_oad_training.models import fuel_tank
from gemseo_oad_training.models import geometry
from gemseo_oad_training.models import mass
from gemseo_oad_training.models import mission
from gemseo_oad_training.models import operating_cost
from gemseo_oad_training.models import take_off
from gemseo_oad_training.models import total_mass

In [2]:
from gemseo import configure_logger

configure_logger()

<RootLogger root (INFO)>

## Problème 1 - Modélisation par surrogate et optimisation

Nous allons créer un modèle de substitution (surrogate model) noté :

$$
\hat{f} : x \mapsto \hat{f}(x) = f(x, u_{\mathrm{default}})
$$

Ce modèle approxime la fonction objectif ainsi que les contraintes du problème de conception, en fonction des paramètres de conception \(x\), avec les conditions d'utilisation fixées à $u_{\mathrm{default}}$.

Une fois ce modèle construit, nous l'utiliserons dans un processus d'optimisation pour :

- **Minimiser** la fonction objectif,
- **Respecter** les contraintes imposées,

en faisant varier les paramètres de conception \(x\).



## The units

The use cases are written using common units.

The models only know STANDARD UNITS which means
that all data provided to the function MUST be expressed in standard units
and that all data retrieved by the function are expressed in standard units.

!!! tip "Convert units"
    The functions `convert_from` and `convert_to` from `gemseo_oad_training.unit` can make the conversion easier.

## The four use cases

| Id  | Fuel type | Engine type | Design range (km) |
|-----|-----------|-------------|-------------------|
| UC1 | Kerosene  | Turbofan    | 5500              |
| UC2 | Hydrogen  | Turbofan    | 5500              |
| UC3 | Hydrogen  | Electrofan  | 5500              |
| UC4 | Battery   | Electrofan  | 500               |

In [3]:
# On veut résoudre le problème 1 avec UC1 et UC2
from gemseo.disciplines.auto_py import AutoPyDiscipline

disciplines = [AutoPyDiscipline(aerodynamic), AutoPyDiscipline(approach), AutoPyDiscipline(battery), AutoPyDiscipline(climb), AutoPyDiscipline(engine), AutoPyDiscipline(fuel_tank), AutoPyDiscipline(geometry), AutoPyDiscipline(mass), AutoPyDiscipline(mission), AutoPyDiscipline(operating_cost), AutoPyDiscipline(take_off), AutoPyDiscipline(total_mass)]

In [4]:
# Create the design space
from gemseo.algos.design_space import DesignSpace
from gemseo_oad_training.unit import convert_from

design_space = DesignSpace()

design_space.add_variable('slst', lower_bound=100000.0, upper_bound=200000.0, value=150000.0)
design_space.add_variable('n_pax', lower_bound=120, upper_bound=180, value=150)
design_space.add_variable('area', lower_bound=100, upper_bound=200, value=180)
design_space.add_variable('ar', lower_bound=5, upper_bound=20, value=9)

In [5]:
# Add the constraints
from gemseo.scenarios.mdo_scenario import MDOScenario
scenario = MDOScenario(disciplines=disciplines, design_space=design_space, objective_name= "mtom",formulation_name='MDF')
scenario.add_constraint('tofl', constraint_type='ineq', value=1900.0, positive=False)
scenario.add_constraint('vapp', constraint_type='ineq', value=convert_from("kt",135.0), positive=False)
scenario.add_constraint('vz', constraint_type='ineq', value=convert_from("ft/min",300.0), positive=True)
scenario.add_constraint('span', constraint_type='ineq', value=40.0, positive=False)
scenario.add_constraint('length', constraint_type='ineq', value=45.0, positive=False)
scenario.add_constraint('fm', constraint_type='ineq', value=0.0, positive=True)

In [6]:
from gemseo import sample_disciplines
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.disciplines.surrogate import SurrogateDiscipline
from numpy import array

training_dataset = sample_disciplines(disciplines, design_space, ["mtom", "tofl", "vapp", "vz", "span", "length", "fm"], algo_name="OT_OPT_LHS", n_samples=50)

test_dataset = sample_disciplines(disciplines, design_space, ["mtom", "tofl", "vapp", "vz", "span", "length", "fm"], algo_name="OT_FULLFACT", n_samples=20)

surrogate_discipline = SurrogateDiscipline("RBFRegressor", training_dataset)

# the optimum values computed by the optimization
surrogate_discipline.execute({"slst": array([150.0]), "n_pax": array([150.0]), "area": array([180.0]), "ar": array([9.0])})

    INFO - 15:23:24: *** Start Sampling execution ***
    INFO - 15:23:24: Sampling
    INFO - 15:23:24:    Disciplines: aerodynamic approach battery climb engine fuel_tank geometry mass mission operating_cost take_off total_mass
    INFO - 15:23:24:    MDO formulation: MDF
    INFO - 15:23:24: Running the algorithm OT_OPT_LHS:


    INFO - 15:23:25:    100%|██████████| 50/50 [00:00<00:00, 56.45 it/sec]
    INFO - 15:23:25: *** End Sampling execution (time: 0:00:00.896264) ***
    INFO - 15:23:25: *** Start Sampling execution ***
    INFO - 15:23:25: Sampling
    INFO - 15:23:25:    Disciplines: aerodynamic approach battery climb engine fuel_tank geometry mass mission operating_cost take_off total_mass
    INFO - 15:23:25:    MDO formulation: MDF
    INFO - 15:23:25: Running the algorithm OT_FULLFACT:
    INFO - 15:23:25:    100%|██████████| 16/16 [00:00<00:00, 48.74 it/sec]s not exist; use 16 samples instead, i.e. the largest 4-th integer power less than 20.
    INFO - 15:23:25: *** End Sampling execution (time: 0:00:00.333820) ***


{'ar': array([9.]),
 'area': array([180.]),
 'n_pax': array([150.]),
 'slst': array([150.]),
 'fm': array([1.05341194]),
 'length': array([37.1880084]),
 'mtom': array([71486.99101902]),
 'span': array([39.11189269]),
 'tofl': array([2250.56021419]),
 'vapp': array([58.6894845]),
 'vz': array([-5.80986584])}

In [7]:
r2 = surrogate_discipline.get_error_measure("R2Measure")
r2.compute_learning_measure(as_dict=True)

{'fm': array([1.]),
 'length': array([1.]),
 'mtom': array([1.]),
 'span': array([1.]),
 'tofl': array([1.]),
 'vapp': array([1.]),
 'vz': array([1.])}

In [8]:
r2.compute_test_measure(test_dataset, as_dict=True)

{'fm': array([0.95156787]),
 'length': array([0.97422938]),
 'mtom': array([0.9627527]),
 'span': array([0.96769612]),
 'tofl': array([0.90289931]),
 'vapp': array([0.96821076]),
 'vz': array([0.94205796])}

In [9]:
rmse = surrogate_discipline.get_error_measure("RMSEMeasure")
rmse.compute_learning_measure(as_dict=True)

{'fm': array([1.13258564e-15]),
 'length': array([5.22140143e-15]),
 'mtom': array([1.21750058e-11]),
 'span': array([1.68669736e-14]),
 'tofl': array([9.36796099e-13]),
 'vapp': array([1.40320947e-14]),
 'vz': array([1.44751099e-14])}

In [10]:
rmse.compute_test_measure(test_dataset, as_dict=True)

{'fm': array([0.23543503]),
 'length': array([0.80266147]),
 'mtom': array([1176.53133244]),
 'span': array([2.75967332]),
 'tofl': array([200.02867414]),
 'vapp': array([1.76394476]),
 'vz': array([1.95017459])}